### Can we classify each phase as relatively low or high anxiety for each subject? ###

In [1]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_apd as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

import lightgbm as lgb
from lightgbm import LGBMClassifier
from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, \
    mean_absolute_error, mean_squared_error, log_loss
from sklearn.model_selection import train_test_split, cross_validate, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)

temp_a, _ = train.Train_APD.get_apd_data_ranking([train.Metrics.BPM], phases=dr.Phases.PHASES_LIST, normalize=False)
idx = temp_a[temp_a["bpm"] > 200].index 
invalid_apd_subjects = set(temp_a["subject"].iloc[idx].tolist())
idx = temp_a[temp_a["bpm"] < 35].index 
invalid_apd_subjects.update(set(temp_a["subject"].iloc[idx].tolist()))

c:\Users\zhoux\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [50]:
models = {
    "LGB": LGBMClassifier(),
    "RF": RandomForestClassifier(random_state=16),
    "XGB": XGBClassifier(random_state=16),
    # "random": None
}

parameters = {
    "LGB": [{
        "objective": ["binary"],
        "num_leaves": [10, 30, 50, 70],
        "metric": ["mean_absolute_error", "mean_squared_error", "binary_logloss"]
    }],
    "RF": [{
        "n_estimators": [10, 50, 100],
        "max_features": ["sqrt"],
        "min_samples_split": [3, 5, 7]
    }],
    "XGB": [{
        "use_label_encoder": [False],
        "objective": ["binary:logistic"],
        "eval_metric": ["error", mean_absolute_error, mean_squared_error, log_loss]
    }],
    # "random": None
}

metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE, 
]

In [51]:
# K-FOLD CROSS-VALIDATION FOR HYPERPARAMETER SELECTION
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)


model_phases = [
    "Baseline_Rest", 
    "BugBox_Relax", "BugBox_Anticipate", "BugBox_Exposure", "BugBox_Break",
    "Speech_Relax", "Speech_Anticipate", "Speech_Exposure", "Speech_Break"
]

threshold = "fixed"

anxiety_label_type = None
x, y = train.Train_APD.get_apd_data_ranking(
    metrics, model_phases, verbose=False, anxiety_label_type=anxiety_label_type, 
    threshold=threshold, normalize=True, combine_phases=False
)
x = x.drop(["phaseId"], axis=1)
inds = pd.isnull(x).any(axis=1).to_numpy().nonzero()[0]
x = x.drop(labels=inds, axis=0).reset_index(drop=True)
y = y.drop(labels=inds, axis=0).reset_index(drop=True)

# drop subjects with noisy data
x = x[~x["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
y = y[~y["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)

if anxiety_label_type is not None:
    x.drop(labels=["anxietyGroup"], axis=1)
    
acc_results = {
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
reports = {
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
best_models = {}

num_iters = 5
get_importance = True
for _ in range(num_iters):
    # HYPERPARAMETER TUNING
    model_data = train.grid_search_cv(
        models, parameters, x, y, by_subject=True, save_metrics=True, is_resample=True,
        get_importance=get_importance, drop_subject=True, test_size=0.2, folds=5
    )

    for model_name in models.keys():
        best_models[model_name] = model_data[model_name]["best_model"]

    # FEATURE SELECTION
    x_train, y_train = model_data["train"]
    features = train.feature_selection(best_models, model_data["cv"], x_train, y_train, n_features=5)

    # TEST USING OPTIMIZED MODELS AND FEATAURES
    x_test, y_test = model_data["test"]
    out = train.train_test_model(best_models, features, x_train, y_train, x_test, y_test)

    for model_name in acc_results:
        acc_results[model_name].append(out[model_name]["performance"][0])
        reports[model_name].append(out[model_name]["performance"][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name]["performance"][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(out[model_name]["performance"][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
    print("\n")

Model LGB, Actual: [0 1], [109  85], Predictions: [0 1], [103  91]
Model RF, Actual: [0 1], [109  85], Predictions: [0 1], [ 83 111]
Model XGB, Actual: [0 1], [109  85], Predictions: [0 1], [116  78]

[('sdnn', 788), ('rmssd', 780), ('bpm', 731), ('lf_rr', 654), ('hf_rr', 629)]


[('rmssd', 0.2341555623428681), ('sdnn', 0.22415488484028406), ('hf_rr', 0.20066685312665905), ('bpm', 0.18269204451750204), ('lf_rr', 0.15833065517268674)]


[('bpm', 0.2270864), ('lf_rr', 0.2149767), ('rmssd', 0.20971696), ('hf_rr', 0.17802972), ('sdnn', 0.17019023)]

Model LGB, Actual: [0 1], [84 93], Predictions: [0 1], [122  55]
Model RF, Actual: [0 1], [84 93], Predictions: [0 1], [133  44]
Model XGB, Actual: [0 1], [84 93], Predictions: [0 1], [135  42]

[('rmssd', 230), ('hf_rr', 213), ('bpm', 211), ('lf_rr', 131), ('sdnn', 115)]


[('bpm', 0.2357522485368522), ('sdnn', 0.22487866020025343), ('rmssd', 0.20036756562970098), ('hf_rr', 0.17060969621861366), ('lf_rr', 0.16839182941457978)]


[('rmssd', 0.2

In [ ]:
# LOAD TRAIN AND TEST DATA -- train on some phases and test on others
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)

# metrics = train.Metrics.ALL
# metrics = train.Metrics.ECG \
    # + train.Metrics.EDA \
    # + train.Metrics.ANKLE + train.Metrics.WRIST

metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    # train.Metrics.HF_RR, 
    # train.Metrics.LF_RR, 
    train.Metrics.IBI, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE, 
]

phases_a = [
    "Baseline_Rest", 
    "BugBox_Relax", 
    "BugBox_Anticipate", 
    # "BugBox_Exposure", 
    # "BugBox_Break",
    "Speech_Relax", 
    "Speech_Anticipate", 
    # "Speech_Exposure", 
    # "Speech_Break"
]

phases_b = [
    # "Baseline_Rest", 
    # "BugBox_Relax", 
    # "BugBox_Anticipate", 
    "BugBox_Exposure", 
    # "BugBox_Break",
    # "Speech_Relax", 
    # "Speech_Anticipate", 
    "Speech_Exposure", 
    # "Speech_Break"
]

models = {
    "SVM": SVC(C=10, gamma=1),  # C=10, gamma=1
    # "KNN": KNeighborsClassifier(n_neighbors=7),
    # "DT": DecisionTreeClassifier(),
    "LogReg": LogisticRegression(max_iter=1000),
    # "Bayes": GaussianNB(),
    "XGB": XGBClassifier(use_label_encoder=False, objective="binary:logistic", eval_metric="logloss")
}

anxiety_label_type = "Anxiety"
# anxiety_label_type = None
test_size = 1.0

x_a, y_a = train.Train_APD.get_apd_data_ranking(metrics, phases_a, verbose=False, anxiety_label_type=anxiety_label_type)
x_b, y_b = train.Train_APD.get_apd_data_ranking(metrics, phases_b, verbose=False, anxiety_label_type=anxiety_label_type)
x_a = x_a.drop(["phaseId"], axis=1)
x_b = x_b.drop(["phaseId"], axis=1)

# drop subjects with noisy data

x_a = x_a[~x_a["subject"].isin(invalid_apd_subjects)]
y_a = y_a[~y_a["subject"].isin(invalid_apd_subjects)]
x_b = x_b[~x_b["subject"].isin(invalid_apd_subjects)]
y_b = y_b[~y_b["subject"].isin(invalid_apd_subjects)]

print(f"y_a:\n{y_a.loc[:, 'label'].value_counts()}")
print(f"y_b:\n{y_b.loc[:, 'label'].value_counts()}")

# x = x[x['subject'] != 8.0]
# y = y[y['subject'] != 8.0]

if anxiety_label_type is not None:
    x_a.drop(labels=["anxietyGroup"], axis=1)
    x_b.drop(labels=["anxietyGroup"], axis=1)

# 0-1 scaling
for i in range(3, len(x_a.columns)):
    data_col = x_a[x_a.columns[i]]
    data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
    x_a[x_a.columns[i]] = data_col
for i in range(3, len(x_b.columns)):
    data_col = x_b[x_b.columns[i]]
    data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
    x_b[x_b.columns[i]] = data_col
    
# make sure subjects from different datasets aren't labeled with the same index
x_b["subject"] = x_b["subject"] + 500

acc_results = {
    "SVM": [],
    "LogReg": [],
    "XGB": []
}
reports = {
    "SVM": [],
    "LogReg": [],
    "XGB": []
}
num_iters = 10
for _ in range(num_iters):
    out = train.Train_Multi_Dataset.train_across_datasets(models, x_a, y_a, x_b, y_b, by_subject=False, save_metrics=True, test_size=test_size)
    for model_name in acc_results:
        acc_results[model_name].append(out[model_name][0])
        reports[model_name].append(out[model_name][1])

for model_name in acc_results.keys():
    acc = np.mean(acc_results[model_name])
    print(f"{model_name} accuracy over {num_iters} rounds: {acc}")
    if acc > 0.4:
        print(f"Model evaluation metrics for {model_name}:")
        p = np.mean([report["precision"] for report in reports[model_name]])
        r = np.mean([report["recall"] for report in reports[model_name]])
        f1 = np.mean([report["f1"] for report in reports[model_name]])
        auc = np.mean([report["auc"] for report in reports[model_name]])
        report = reports[model_name]
        print(f"Precision: {p}\nRecall: {r}\nF1-score: {f1}\nAUC score: {auc}")
    print("")
print("\n")